## Chapter 3: Low-Rank Adaptation (LoRA)

### Spoilers

In this chapter, we will:

- Understand what a low-rank adapter is and why it’s useful
- Prepare the quantized model for training
- Use `peft` to create and attach adapters to a base model
- Discuss configuration options for targeting layers for training

### Setup

In [1]:
# If you're running on Colab
!pip install datasets bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 482.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

In [ ]:
# If you're running on runpod.io's Jupyter Template
#!pip install transformers peft huggingface-hub accelerate safetensors pandas matplotlib

### Imports

In [2]:
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from numpy.linalg import matrix_rank
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

### The Goal

We attach adapters to the huge linear layers in an LLM to drastically reduce the number of trainable parameters. We can easily shrink the number of trainable parameters down to less than 1% of their original number. By reducing both computation (fewer gradients to compute) and memory footprint (fewer parameters tracked by the optimizer), we achieve significant efficiency gains. Keep in mind, however, that low-rank adapters are unlikely to match the performance of full-model tuning, and their effectiveness may vary depending on the base model and the task.

### Pre-Reqs

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/matmul.png?raw=True)
<center>Figure 3.1 - Matrix multiplication</center>

### Low-Rank Adaptation in a Nutshell

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/two_matrices.png?raw=True)
<center>Figure 3.2 - Multiplying two low-rank matrices</center>

In [3]:
base_layer = nn.Linear(1024, 1024, bias=False)
base_layer.weight.shape, base_layer.weight.numel()

(torch.Size([1024, 1024]), 1048576)

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/lowrank_matrices.png?raw=True)
<center>Figure 3.3 - Frozen weights and low-rank matrices</center>

In [4]:
torch.manual_seed(11)
r = 8
layer_A = nn.Linear(base_layer.in_features, r, bias=False)
layer_B = nn.Linear(r, base_layer.out_features, bias=False)
layer_A, layer_B

(Linear(in_features=1024, out_features=8, bias=False),
 Linear(in_features=8, out_features=1024, bias=False))

In [5]:
layer_A.weight.numel(), layer_B.weight.numel()

(8192, 8192)

In [6]:
composite = layer_B.weight @ layer_A.weight
composite.shape, composite.numel()

(torch.Size([1024, 1024]), 1048576)

In [7]:
matrix_rank(composite.detach().numpy())

8

$$
\Large
\text{output} = X @ (W + B @ A)^T
$$
<center>Equation 3.1 - Adding the resulting product to the weights</center>

In [7]:
torch.manual_seed(19)
batch = torch.randn(1, 1024)

batch @ (base_layer.weight.data + layer_B.weight @ layer_A.weight).T

tensor([[-0.3740,  0.0485, -1.2761,  ...,  0.5605,  0.1853,  0.5227]],
       grad_fn=<MmBackward0>)

$$
\Large
\text{output} = \underbrace{X @ W^T}_{O_W} + \underbrace{X @ (B @ A)^T}_{O_{AB}}
$$
<center>Equation 3.2 - Using two forward passes</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/forward.png?raw=True)
<center>Figure 3.4 - Using two forward passes</center>

In [8]:
regular_output = batch @ base_layer.weight.data.T
additional_output = batch @ (layer_B.weight @ layer_A.weight).T
regular_output, additional_output

(tensor([[-0.3383,  0.0255, -0.8154,  ...,  0.8525, -0.0091,  0.0186]]),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>))

$$
\Large
\text{additional} = X @ (B @ A)^T = \underbrace{\underbrace{(X @ A^T)}_{O_A} @ B^T}_{O_{AB}}
$$
<center>Equation 3.3 - Chaining the adapter’s forward passes</center>

In [9]:
out_A = (batch @ layer_A.weight.T)
additional_output = out_A @ layer_B.weight.T
additional_output

tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
       grad_fn=<MmBackward0>)

In [10]:
regular_output = base_layer(batch)
out_A = layer_A(batch)
additional_output = layer_B(out_A)
output = regular_output + additional_output
regular_output, additional_output, output

(tensor([[-0.3383,  0.0255, -0.8154,  ...,  0.8525, -0.0091,  0.0186]],
        grad_fn=<MmBackward0>),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>),
 tensor([[-0.3740,  0.0485, -1.2761,  ...,  0.5605,  0.1853,  0.5227]],
        grad_fn=<AddBackward0>))

$$
\Large
\text{output} = X @ W^T + \frac{\alpha}{r}\left[X @ (B @ A)^T\right]
$$
<center>Equation 3.4 - LoRA’s alpha</center>

In [11]:
alpha = 2*r
output = regular_output + (alpha / r) * additional_output
output

tensor([[-0.4097,  0.0715, -1.7368,  ...,  0.2684,  0.3796,  1.0268]],
       grad_fn=<AddBackward0>)

### The Road So Far

In [13]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

model_q4 = AutoModelForCausalLM.from_pretrained("facebook/opt-350m",
                                                device_map='auto',
                                                torch_dtype=compute_dtype,
                                                quantization_config=nf4_config)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Parameter Types and Gradients

****
**Summary of "Parameter Types and Gradients"**
- quantization only freezes the linear layers that have been quantized
- after quantization, a model can be prepared using the `prepare_model_for_kbit_training()` function
  - it freezes **all** layers
  - it casts every non-quantized 16-bit layer to FP32 to improve training
  - it enables gradient checkpointing
- you'll be able to unfreeze layers of your choice later on using the LoRA configuration
****

In [14]:
def trainable_parms(model):
    parms = [(name, param.dtype) for name, param in model.named_parameters() if param.requires_grad]
    return parms

trainable_parms(model_q4.model)

[('decoder.embed_tokens.weight', torch.float32),
 ('decoder.embed_positions.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.0.final_layer_norm.weight', torch.float32),
 ('decoder.layers.0.final_layer_norm.bias', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.1.final_layer_norm.weight', torch.float32),
 ('decoder.layers.1.final_layer_norm.bias', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.2.final_layer_norm.weight', torch.float32),
 ('decoder.layers.2.final_layer_norm.bias', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.bias', torch.float32),
 ('decoder.la

#### `prepare_model_for_kbit_training()`

In [15]:
prepared_model = prepare_model_for_kbit_training(model_q4,
                                        use_gradient_checkpointing=True,
                                        gradient_checkpointing_kwargs={'use_reentrant': False})
prepared_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
       

In [16]:
trainable_parms(prepared_model)

[]

In [17]:
def parms_of_dtype(model, dtype=torch.float32):
    parms = [name for name, param in model.named_parameters() if param.dtype == dtype]
    return parms

In [18]:
parms_of_dtype(prepared_model)

['model.decoder.embed_tokens.weight',
 'model.decoder.embed_positions.weight',
 'model.decoder.layers.0.self_attn.k_proj.bias',
 'model.decoder.layers.0.self_attn.v_proj.bias',
 'model.decoder.layers.0.self_attn.q_proj.bias',
 'model.decoder.layers.0.self_attn.out_proj.bias',
 'model.decoder.layers.0.self_attn_layer_norm.weight',
 'model.decoder.layers.0.self_attn_layer_norm.bias',
 'model.decoder.layers.0.fc1.bias',
 'model.decoder.layers.0.fc2.bias',
 'model.decoder.layers.0.final_layer_norm.weight',
 'model.decoder.layers.0.final_layer_norm.bias',
 'model.decoder.layers.1.self_attn.k_proj.bias',
 'model.decoder.layers.1.self_attn.v_proj.bias',
 'model.decoder.layers.1.self_attn.q_proj.bias',
 'model.decoder.layers.1.self_attn.out_proj.bias',
 'model.decoder.layers.1.self_attn_layer_norm.weight',
 'model.decoder.layers.1.self_attn_layer_norm.bias',
 'model.decoder.layers.1.fc1.bias',
 'model.decoder.layers.1.fc2.bias',
 'model.decoder.layers.1.final_layer_norm.weight',
 'model.decode

In [19]:
prepared_model.get_memory_footprint()/1e6

264.15104

### PEFT

"_🤗 PEFT (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters because it is prohibitively costly. PEFT methods only fine-tune a small number of (extra) model parameters - significantly decreasing computational and storage costs - while yielding performance comparable to a fully fine-tuned model. This makes it more accessible to train and store large language models (LLMs) on consumer hardware._

_PEFT is integrated with the Transformers, Diffusers, and Accelerate libraries to provide a faster and easier way to load, train, and use large models for inference._"

****
**Summary of "PEFT"**
- the basic configuration below should work well in many cases
```python
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)
```
- ranks of 8, 16, or 32 are typical, but using higher values shouldn’t significantly impact the model’s memory footprint.
- the scaling factor, `lora_alpha` is typically twice the rank.
- if your model has `Conv1D` layers, add `fan_in_fan_out=True` to your configuration
- if your model was recently released, you may need to specify the `target_modules` manually
  - typically, use the names of the massive linear layers in the attention module.
- by default, only the adapters are trainable
  - if you'd like to train other layers, such as layer norms, add them to the `modules_to_save` argument
  - if you're adding your own tokens to the tokenizer, you'll need to also train vocabulary-related layers such as embeddings and the model's head
****

In [20]:
lora_config = LoraConfig()
lora_config

LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules=None, exclude_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [21]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

### `target_modules`

Since there are new models and architectures being released on a weekly basis, chances are that there is no preconfigured list of target layers in your currently installed version of the PEFT library. In this case, you’ll be greeted with the following error:

***
`ValueError: Please specify `target_modules` in `peft_config``
***

Once you have the names, you can use yet another configuration argument: target_modules, which is either
the name or a list of the names of the modules to which you want to apply the adapters.

**Supported Models**

If you'd like to check if a given model's architecture is already supported by the installed version of the `peft` package, you can do the following:

In [22]:
from peft.utils.constants import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING.keys()

dict_keys(['t5', 'mt5', 'bart', 'gpt2', 'bloom', 'blip-2', 'opt', 'gptj', 'gpt_neox', 'gpt_neo', 'bert', 'roberta', 'xlm-roberta', 'electra', 'deberta-v2', 'deberta', 'layoutlm', 'llama', 'chatglm', 'gpt_bigcode', 'mpt', 'RefinedWebModel', 'RefinedWeb', 'falcon', 'btlm', 'codegen', 'mistral', 'mixtral', 'stablelm', 'phi', 'gemma', 'gemma2', 'qwen2'])

In [23]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['phi']

['q_proj', 'v_proj', 'fc1', 'fc2']

#### The PEFT Model

In [24]:
peft_model = get_peft_model(prepared_model, config, adapter_name='default')
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTSdpaAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out

In [25]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['opt']

['q_proj', 'v_proj']

In [26]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj
lin

lora.Linear4bit(
  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=8, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [27]:
peft_model.print_trainable_parameters()

trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [28]:
trainable_parms(peft_model.base_model.model)

[('model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_B.default.weight'

#### `modules_to_save`

In [30]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm']
)

In [ ]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [32]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 884,736 || all params: 332,081,152 || trainable%: 0.2664


#### Embeddings

In [33]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm', 'embed_tokens']
)

In [34]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [36]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 26,624,000 || all params: 357,820,416 || trainable%: 7.4406


In [37]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['embed_tokens', 'q_proj', 'v_proj']
)

In [38]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [39]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 1,192,704 || all params: 358,128,384 || trainable%: 0.3330


In [40]:
lin = peft_model.base_model.model.model.decoder.embed_tokens
lin

lora.Embedding(
  (base_layer): Embedding(50272, 512, padding_idx=1)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict()
  (lora_B): ModuleDict()
  (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 8x50272 (cuda:0)])
  (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 512x8 (cuda:0)])
  (lora_magnitude_vector): ModuleDict()
)

#### Managing Adapters

In [ ]:
peft_model.load_adapter('dvgodoy/opt-350m-lora-yoda', adapter_name='yoda')
lora_A = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj.lora_A
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.add_adapter(adapter_name='third', peft_config=config)
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
  (third): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.delete_adapter(adapter_name='third')
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.peft_config.keys()

dict_keys(['default', 'yoda'])

In [ ]:
peft_model.active_adapter

'default'

In [ ]:
peft_model.set_adapter('yoda')
peft_model.active_adapter

'yoda'

```python
with peft_model.disable_adapter():
    original_outputs = peft_model(inputs)

original_outputs = peft_model.base_model(inputs)
```

In [ ]:
peft_model.merge_adapter(adapter_names=['yoda'])
lora_A

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.unload()
peft_model.base_model.model.model.decoder.layers[0].self_attn

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              )
              (activation_fn): ReLU()
              (self_attn_layer_norm): LayerNorm((1024,

### Coming Up in "Fine-Tuning LLMs"

Low-rank adapters saved the day by swooping in and enabling fast and cheap fine-tuning for LLMs. These humongous models, although powerful, are masters of a single trade—predicting the next token—thus remaining limited by the structure of their inputs. A new kind of input must be developed to enable these creatures to chat. Learn more about the incredible tale of chat templates in the next chapter of "Fine-Tuning LLMs."